In [1]:
# with open('the-verdict.txt', 'r', encoding='utf-8') as f:
#     lines = [line for line in f if line.strip()]

# with open("the-verdict.txt", "w", encoding="utf-8") as f:
#     f.writelines(lines)

In [2]:
with open('the-verdict.txt','r', encoding= 'utf-8') as f:
    raw_text = f.read();
print("total chars: ", len(raw_text))
print(raw_text[:99])

total chars:  20398
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re
text = "Hello, world. Is this-- a test?"
res = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in res if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


Apply this tokenization scheme to the entire dataset

In [4]:
preprocess = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed_text = [item for item in preprocess if item.strip()]
print(len(preprocessed_text))

4690


In [5]:
all_words= sorted(set(preprocessed_text))

vocab_size = len(all_words)
vocab = {s:num for num, s in enumerate(all_words)}
print(vocab_size)

1130


In [6]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}
        
    def encode(self, text):
        preprocess = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocess = [item for item in preprocess if item.strip()]
        ids = [self.str_to_int[s] for s in preprocess]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

Now we add 2 more sepcial characters if we encounter some token not in our dict and for indicating end of some particular context

In [7]:
tokens = sorted(set(preprocessed_text))
tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {s:i for i, s in enumerate(tokens)}
print(len(vocab))

1132


In [8]:
class SimpleTokenizer2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
        
    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids = [self.str_to_int[i] for i in preprocessed]
        return ids
    
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text
        
        

Byte pair encoding

In [9]:
import tiktoken
tokenizer = tiktoken.get_encoding('gpt2')
text = (
"Hello, do you like tea? <|endoftext|> In the sunlit terraces"
"of someunknownPlace."
)
nums = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(nums)
decodedText = tokenizer.decode(nums)
print(decodedText)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [10]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5064


In [11]:
enc_sample = enc_text[50:]

In [12]:
context_size = 4;
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "------>", desired)
    

[290] ------> 4920
[290, 4920] ------> 2241
[290, 4920, 2241] ------> 287
[290, 4920, 2241, 287] ------> 257


In [13]:
for i in range(1, context_size + 1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(tokenizer.decode(context), '------>', tokenizer.decode([desired]))

 and ------>  established
 and established ------>  himself
 and established himself ------>  in
 and established himself in ------>  a


In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

In [15]:
class GPTDataset(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk= token_ids[i:i + max_length]
            target_chunk= token_ids[i + 1:i + max_length + 1];
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
            
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]

In [16]:
def create_dataloader_v1(txt, batch_size= 4, max_length= 256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    tokenizer=tiktoken.get_encoding('gpt2')
    dataset=GPTDataset(txt, tokenizer,max_length, stride)
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
        
    )
    return dataloader

In [17]:
dataloader=create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [18]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


2.7 word embeddings

In [19]:
vocab_size=6
embd_dim=3
embedding_layer= torch.nn.Embedding(vocab_size, embd_dim)
#We can just index the tokens id and basically perform a lookup operation
print(embedding_layer(torch.tensor([3])))

tensor([[-0.0440,  0.5022, -0.1614]], grad_fn=<EmbeddingBackward0>)


In [20]:
vocab_size=50257
output_dim=256
token_embedding_layer=torch.nn.Embedding(vocab_size, output_dim)


In [21]:
max_length=4
dataloader= create_dataloader_v1(raw_text,batch_size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)
print('Token ids: ', inputs)
print('Shape', inputs.shape)

Token ids:  tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Shape torch.Size([8, 4])


In [ ]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


2.8 pos embeddings

In [27]:

context_length= max_length
pos_embedding_layer= torch.nn.Embedding(context_length, output_dim)
pos_embeddings=pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


In [28]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])


Chapter 3

In [ ]:
inputs = torch.tensor(
[[0.43, 0.15, 0.89], # Your (x^1)
[0.55, 0.87, 0.66], # journey (x^2)
[0.57, 0.85, 0.64], # starts (x^3)
[0.22, 0.58, 0.33], # with (x^4)
[0.77, 0.25, 0.10], # one (x^5)
[0.05, 0.80, 0.55]] # step (x^6)
)